In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 200)

In [2]:
movies = pd.read_csv('./datasets/movies.csv')
links = pd.read_csv('./datasets/links.csv')
ratings = pd.read_csv('./datasets/ratings.csv')
tags = pd.read_csv('./datasets/tags.csv')

In [3]:
print('movies')
print(movies.shape)
print(movies.dtypes)
print()
print('movies null:')
print(movies.isnull().sum())
print()
print('links')
print(links.shape)
print(links.dtypes)
print()
print('links null:')
print(links.isnull().sum())
print()
print('ratings')
print(ratings.shape)
print(ratings.dtypes)
print()
print('ratings null:')
print(ratings.isnull().sum())
print()
print('tags')
print(tags.shape)
print(tags.dtypes)
print()
print('tags null:')
print(tags.isnull().sum())

movies
(9742, 3)
movieId     int64
title      object
genres     object
dtype: object

movies null:
movieId    0
title      0
genres     0
dtype: int64

links
(9742, 3)
movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

links null:
movieId    0
imdbId     0
tmdbId     8
dtype: int64

ratings
(100836, 4)
userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

ratings null:
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

tags
(3683, 4)
userId        int64
movieId       int64
tag          object
timestamp     int64
dtype: object

tags null:
userId       0
movieId      0
tag          0
timestamp    0
dtype: int64


In [4]:
# to make a copy of a dataframe to practice with, use .copy(deep = True)
# deep=True means changing one does not affect the other. deep=False is the reverse

# df2 = df.copy(deep=True)

### Movies and Links

In [5]:
df = pd.merge(movies, links)
df.columns = df.columns.str.lower()
df.rename(columns={'movieid':'movie_id', 'imdbid':'imdb_id', 'tmdbid':'tmdb_id'}, inplace=True)

In [6]:
# dropping 8 null values from links.csv
# df.isnull().sum()
# df['tmdb_id'].loc[df['tmdb_id'].isnull()==True]
df.dropna(inplace=True)

In [7]:
# dataframe with all movies whose title has an extra space at the end
spaces = df.loc[df['title'].str[-1] == ' ']

# getting rid of that extra space
for i in spaces['title']:
    df.replace(to_replace = {'title': i},
                value = i[:-1],
                inplace = True)

# fixing an individual row that was formatted differently
df.replace(to_replace = {'title':'Death Note: Desu nôto (2006–2007)'},
           value = 'Death Note: Desu nôto (2006)',
           inplace = True)

In [8]:
# creating a new column for release year
release_years = []
for i in df['title']:
    release_years.append(i[-5:-1])
df['release_year'] = release_years

# getting rid of the year from the title column for
# the majority of movies which have it listed there

majority = df.loc[df['title'].str[-1] == ')']
anomalies = df.loc[df['title'].str[-1] != ')']

for i in majority['title']:
    df.replace(to_replace = {'title': i},
                value = i[:-7],
                inplace = True)

# fixing another individual row that is formatted differently
df.replace(to_replace = {'title':'Angst '},
           value = 'Angst',
           inplace = True)

In [9]:
# taking care of the anomalies manually.
# I looked up the release years on IMDB, because figuring out how to code a web scrape on imdb
# would have taken me more time than Doctor Strange has access to.

df['release_year'][6059] = 1993
df['release_year'][9031] = 2018
df['release_year'][9091] = 2015
df['release_year'][9138] = 1980
df['release_year'][9179] = 2016
df['release_year'][9259] = 2016
df['release_year'][9367] = 2016
df['release_year'][9448] = 2016
df['release_year'][9514] = 1980
df['release_year'][9515] = 2017
df['release_year'][9525] = 2017
df['release_year'][9611] = 2011

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCo

In [10]:
# creating new column for decade

# converting values to strings to make them indexable 
df['release_year'] = df['release_year'].astype(str)

decades_list = []

for i in df['release_year']:
    if i[:3] == '190':
        decades_list.append('1900s')
    elif i[:3] == '191':
        decades_list.append('1910s')
    elif i[:3] == '192':
        decades_list.append('1920s')
    elif i[:3] == '193':
        decades_list.append('1930s')
    elif i[:3] == '194':
        decades_list.append('1940s')
    elif i[:3] == '195':
        decades_list.append('1950s')
    elif i[:3] == '196':
        decades_list.append('1960s')
    elif i[:3] == '197':
        decades_list.append('1970s')
    elif i[:3] == '198':
        decades_list.append('1980s')
    elif i[:3] == '199':
        decades_list.append('1990s')
    elif i[:3] == '200':
        decades_list.append('2000s')
    elif i[:3] == '201':
        decades_list.append('2010s')

df['decade'] = decades_list

In [11]:
# converting values back to numerical type
df['release_year'] = df['release_year'].astype(int)

In [12]:
genres_list = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
               'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
               'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


# creating dummy columns for the 18 main genres

for i in genres_list:
    df[i] = np.where(df['genres'].str.contains(i), 1, 0)

### Ratings

In [13]:
# lowercase, renaming, and dropping timestamp column

ratings.columns = ratings.columns.str.lower()
ratings.rename(columns={'movieid':'movie_id', 'userid':'user_id'}, inplace=True)
ratings.drop(columns = ['timestamp'], inplace=True)

In [14]:
# adding new columns to main df for mean, max, min, median, and number of ratings for each movie

avg_ratings = []
for i in df['movie_id']:
    avg_rating = ratings.loc[ratings['movie_id']==i]['rating'].mean()
    avg_ratings.append(avg_rating)
df['avg_rating'] = avg_ratings

max_ratings = []
for i in df['movie_id']:
    max_rating = ratings.loc[ratings['movie_id']==i]['rating'].max()
    max_ratings.append(max_rating)
df['max_rating'] = max_ratings

min_ratings = []
for i in df['movie_id']:
    min_rating = ratings.loc[ratings['movie_id']==i]['rating'].min()
    min_ratings.append(min_rating)
df['min_rating'] = min_ratings

med_ratings = []
for i in df['movie_id']:
    med_rating = ratings.loc[ratings['movie_id']==i]['rating'].median()
    med_ratings.append(med_rating)
df['med_rating'] = med_ratings

numb_ratings = []
for i in df['movie_id']:
    numb_rating = ratings.loc[ratings['movie_id']==i]['rating'].count()
    numb_ratings.append(numb_rating)
df['numb_rating'] = numb_ratings

In [15]:
df.head()

,movie_id,title,genres,imdb_id,tmdb_id,release_year,decade,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,avg_rating,max_rating,min_rating,med_rating,numb_rating
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,1990s,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,3.920930,5.0,0.5,4.0,215
1,2,Jumanji,Adventure|Children|Fantasy,113497,8844.0,1995,1990s,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.431818,5.0,0.5,3.5,110
2,3,Grumpier Old Men,Comedy|Romance,113228,15602.0,1995,1990s,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,3.259615,5.0,0.5,3.0,52
3,4,Waiting to Exhale,Comedy|Drama|Romance,114885,31357.0,1995,1990s,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,2.357143,3.0,1.0,3.0,7
4,5,Father of the Bride Part II,Comedy,113041,11862.0,1995,1990s,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.071429,5.0,0.5,3.0,49


# TO DO next:
> make dummy columns for the 18 genres outlined on the grouplens.org page

> categorize release years into groups

> look for correlations among at least these five columns: genre, avg_rating, # of ratings, release Year, and whether or not there's a tag

In [ ]:
ratings.head(20)

In [ ]:
ratings.corrwith(df['avg_rating'])

In [ ]:
ratings.corrwith(df['release_year'])

In [ ]:
df['genres'].value_counts(normalize = True)

In [ ]:
df[df['genres'].str.contains('Comedy')]['genres'].value_counts()

In [ ]:
plt.figure(figsize = (8,12))
sns.heatmap(df.corr()[['avg_rating']].sort_values('avg_rating'),
                                                   annot = True,
                                                   seamap = 'coolwarm',
                                                   vmin = -1,
           vmax = 1)